### Get the data

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train.shape

(60000, 28, 28)

In [2]:
x_train[:1,:10]

array([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,
           0,   0,   0

In [3]:
# preprocess
x_train, x_test = x_train / 255.0, x_test / 255.0 # scale image features


### Build model

In [4]:
num_inputs=28*28 # mnist format
num_hidden=300
num_hidden2=100
n_outputs=10 # mnist target classes, numbers from 0-9

In [50]:
with tf.name_scope("inputs"):
    input_tensor=tf.placeholder(shape=[None,num_inputs],dtype=float,name="image_input")
    target=tf.placeholder(shape=[None],dtype=tf.int64, name="target")

with tf.name_scope("dnn"):
    layer1=tf.layers.dense(input_tensor,num_hidden,activation="relu")
    layer2=tf.layers.dense(layer1,num_hidden2,activation="relu")
    logits=tf.layers.dense(layer2, n_outputs,activation=None)

with tf.name_scope("loss"):
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=target)
    loss=tf.reduce_mean(cross_entropy) # mean within batch
    
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(0.01)
    train_operation=optimizer.minimize(loss)

with tf.name_scope("prediction"):
    predicted=tf.argmax(logits,axis=1)  # since softmax uses a logistic function (mononotically increasing) 
                                        # the argmax of the inputs is the same as the argmax of softmax(inputs) 
with tf.name_scope("evaluation"):
    hit=tf.math.equal(predicted,target)
    accuracy=tf.reduce_mean(tf.cast(hit,tf.float32))
    

In [51]:
# reshape data for usage
train_target=y_train.reshape(-1)
test_target=y_test.reshape(-1)
train_data=x_train.reshape(x_train.shape[0],-1)
test_data=x_test.reshape(x_test.shape[0],-1)
train_data.shape,train_target.shape

((60000, 784), (60000,))

### Train model

In [52]:
import numpy as np
import time
import os

num_epoches=1
batch_size=64 # x_train.shape[0]

#os.makedirs("model/run",exist_ok=True)
saver=tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for e in range(num_epoches):
        print("Epoch %i" %e)
        for iteration in range(x_train.shape[0]//batch_size):
            sample_idx=np.random.randint(0,x_train.shape[0], batch_size)
            sess.run(train_operation,feed_dict={target:train_target[sample_idx],
                                                  input_tensor:train_data[sample_idx]})
        accuracy_training=sess.run(accuracy,feed_dict={target:train_target,
                                                      input_tensor:train_data})
        accuracy_test=sess.run(accuracy,feed_dict={target:test_target,
                                                      input_tensor:test_data})
        print(accuracy_training,accuracy_test)
    prediction_training=sess.run(predicted,feed_dict={input_tensor:test_data})
    saver.save(sess,save_path="./model.ckpt")
    

Epoch 0
0.8889667 0.8968


In [53]:
prediction_training

array([7, 2, 1, ..., 4, 5, 6])

In [65]:
# load model (for example later, in different environment)

with tf.Session() as sess:
    tf.train.Saver().restore(sess,"./model.ckpt")
    
    # test that model gives same result in new environment as in training
    prediction_serving_environment=sess.run(predicted,feed_dict={input_tensor:test_data}) # use some new data (I reuse test data)
    assert np.array_equal(prediction_training,prediction_serving_environment)

INFO:tensorflow:Restoring parameters from ./model.ckpt


In [17]:
! ls model/run/checkpoint/


ls: cannot access 'model/run/checkpoint/': Not a directory
